In [1]:
pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
from bs4 import BeautifulSoup

# Define the URL template for the product listing pages
url_template = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_{}"

# Number of pages to scrape
num_pages = 20

# Initialize lists to store the extracted data
product_urls = []
product_names = []
product_prices = []
product_ratings = []
product_reviews = []

# Scrape the product listing pages
for page in range(1, num_pages + 1):
    url = url_template.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract the required information from the page
    results = soup.find_all("div", {"data-component-type": "s-search-result"})

    for result in results:
        # Extract product URL
        product_link = result.find("a", {"class": "a-link-normal s-no-outline"})
        if product_link:
            product_url = "https://www.amazon.in" + product_link["href"]
            product_urls.append(product_url)

        # Extract product name
        product_name = result.find("span", {"class": "a-size-medium a-color-base a-text-normal"})
        if product_name:
            product_names.append(product_name.text.strip())

        # Extract product price
        product_price = result.find("span", {"class": "a-offscreen"})
        if product_price:
            product_prices.append(product_price.text.strip())

        # Extract product rating
        product_rating = result.find("span", {"class": "a-icon-alt"})
        if product_rating:
            product_ratings.append(product_rating.text.strip())

        # Extract number of reviews
        product_review = result.find("span", {"class": "a-size-base"})
        if product_review:
            product_reviews.append(product_review.text.strip())

# Print the extracted data
for i in range(len(product_urls)):
    print("Product URL:", product_urls[i])
    print("Product Name:", product_names[i])
    print("Product Price:", product_prices[i])
    print("Product Rating:", product_ratings[i])
    print("Number of Reviews:", product_reviews[i])
    print("---------------------")

Product URL: https://www.amazon.in/Wesley-Milestone-Waterproof-Backpack-Business/dp/B084JGJ8PF
Product Name: Wesley Milestone 2.0 Casual Waterproof Laptop Backpack/Office Bag/School Bag/College Bag/Business Bag/Travel Backpack (Dimensions:13x18 inches) (Compatible with 39.62cm(15.6inch laptop) 30 L (Blue & black)
Product Price: ₹598
Product Rating: 4.3 out of 5 stars
Number of Reviews: 10,944
---------------------
Product URL: https://www.amazon.in/American-Tourister-AMT-SCH-02/dp/B07CJCGM1M
Product Name: American Tourister 32 Ltrs Black Casual Backpack (AMT FIZZ SCH BAG 02 - BLACK)
Product Price: ₹1,199
Product Rating: 4.0 out of 5 stars
Number of Reviews: More Like This
---------------------
Product URL: https://www.amazon.in/Skybags-Brat-Black-Casual-Backpack/dp/B08Z1HHHTD
Product Name: Skybags Brat Black 46 Cms Casual Backpack
Product Price: ₹669
Product Rating: 4.1 out of 5 stars
Number of Reviews: More Like This
---------------------
Product URL: https://www.amazon.in/ADISA-Lapto

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Function to scrape product details from a URL
def scrape_product_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract description
    product_description = soup.find("meta", attrs={"name": "description"})
    description = product_description["content"] if product_description else "N/A"

    # Extract ASIN
    asin_element = soup.find("th", text="ASIN")
    asin = asin_element.find_next("td").text.strip() if asin_element else "N/A"

    # Extract product description
    product_desc_element = soup.find("h2", attrs={"id": "productTitle"})
    product_desc = product_desc_element.text.strip() if product_desc_element else "N/A"

    # Extract manufacturer
    manufacturer_element = soup.find("a", attrs={"id": "bylineInfo"})
    manufacturer = manufacturer_element.text.strip() if manufacturer_element else "N/A"

    return description, asin, product_desc, manufacturer


# Define the URL template for the product listing pages
url_template = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_{}"

# Number of pages to scrape (minimum 20)
num_pages = 20

# Initialize lists to store the extracted data
product_urls = []
product_names = []
product_prices = []
product_ratings = []
product_reviews = []
descriptions = []
asins = []
product_descriptions = []
manufacturers = []

# Scrape the product listing pages
for page in range(1, num_pages + 1):
    url = url_template.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract product details from the page
    products = soup.find_all("div", class_="sg-col-inner")

    for product in products:
        # Extract product URL
        product_link = product.find("a", class_="a-link-normal s-no-outline")
        if product_link:
            product_url = "https://www.amazon.in" + product_link["href"]
            product_urls.append(product_url)

            # Extract product name, price, rating, and number of reviews
            product_name = product_link.find("span", class_="a-size-medium a-color-base a-text-normal")
            product_price = product.find("span", class_="a-offscreen")
            product_rating = product.find("span", class_="a-icon-alt")
            product_review = product.find("span", class_="a-size-base")

            product_names.append(product_name.text.strip() if product_name else "N/A")
            product_prices.append(product_price.text.strip() if product_price else "N/A")
            product_ratings.append(product_rating.text.strip() if product_rating else "N/A")
            product_reviews.append(product_review.text.strip() if product_review else "N/A")

    # Pause for a short while before hitting each product URL to avoid overwhelming the server
    time.sleep(2)

# Scrape additional information from each product URL
for url in product_urls:
    description, asin, product_desc, manufacturer = scrape_product_details(url)
    descriptions.append(description)
    asins.append(asin)
    product_descriptions.append(product_desc)
    manufacturers.append(manufacturer)

# Prepare data for CSV export
data = zip(product_urls, product_names, product_prices, product_ratings, product_reviews,
           descriptions, asins, product_descriptions, manufacturers)

# Export data to CSV file
csv_filename = "product_data.csv"
with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["URL", "Name", "Price", "Rating", "Reviews",
                     "Description", "ASIN", "Product Description", "Manufacturer"])
    writer.writerows(data)

print("Data exported to", csv_filename)

<ipython-input-3-6ee376e12f94>:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  asin_element = soup.find("th", text="ASIN")


Data exported to product_data.csv
